In [2]:
import psycopg2


In [ ]:

my_conn = psycopg2.connect(
    dbname="student_management", #ur database name
    user="", #ur username
    password="", #ur password
    port="", #ur port number
    host="localhost",
)
cur = my_conn.cursor()
print("Database connected successfully.")

Database connected successfully.


In [4]:
cur= my_conn.cursor()

### to create a new database

In [5]:
my_conn.autocommit = True
cur = my_conn.cursor()
cur.execute("CREATE DATABASE student_management;")

DuplicateDatabase: database "student_management" already exists


In [6]:
cur= my_conn.cursor()

### show databases available


In [7]:
cur.execute("SELECT datname FROM pg_database WHERE datistemplate = false;")
for db in cur.fetchall():
    print(db[0])


postgres
DEPi
Student
ds_new
student_management


### creating tables

In [ ]:
cur.execute("""
            CREATE TABLE students(
                student_id SERIAL PRIMARY KEY,
                name VARCHAR(40),
                email VARCHAR(30),
                phone VARCHAR(20))
""")
print("Table 'students' created successfully")

Table 'students' created successfully


In [10]:
cur.execute("""
            CREATE TABLE courses(
                course_id SERIAL PRIMARY KEY,
                course_name VARCHAR(15),
                credits INT)        
""")
print("Table 'courses' created successfully")

Table 'courses' created successfully


### altering value for type character varying(15) to (30)

In [18]:
cur.execute("""
ALTER TABLE courses
ALTER COLUMN course_name TYPE VARCHAR(30)
""")
my_conn.commit()

In [11]:
cur.execute("""
            CREATE TABLE enrollments(
                enrollment_id SERIAL PRIMARY KEY,
                student_id INT REFERENCES students(student_id),
                course_id INT REFERENCES courses(course_id),
                grade VARCHAR(5))        
""")
print("Table 'courses' created successfully")

Table 'courses' created successfully


### insertions

In [13]:
sql = "INSERT INTO students (name,email,phone)VALUES (%s, %s, %s)"
data =[
    ('Omar Khaled', 'omar.khaled@example.com', '01098765432'),
    ('Sara Nabil', 'sara.nabil@example.com', '01122334455'),
    ('Mahmoud Adel', 'mahmoud.adel@example.com', '01233445566'),
    ('Nour El-Sayed', 'nour.elsayed@example.com', '01044556677'),
    ('Youssef Hassan', 'youssef.hassan@example.com', '01577889900')
]
cur.executemany(sql, data)
my_conn.commit()

print(cur.rowcount,"the new record was inserted")

5 the new record was inserted


In [17]:
sql = "INSERT INTO courses (course_name,credits)VALUES (%s, %s)"
data =[
    ('Mathematics', 3),
    ('Computer Science', 4),
    ('History', 2)
]
cur.executemany(sql, data)
my_conn.commit()

print(cur.rowcount,"the new record was inserted")

3 the new record was inserted


In [20]:
sql = "INSERT INTO enrollments (student_id,course_id,grade)VALUES (%s,%s, %s)"
data =[
    (1, 1, 'A'),
    (1, 2, 'B'),
    (2, 3, 'A'),
    (3, 2, 'C')
]
cur.executemany(sql, data)
my_conn.commit()

print(cur.rowcount,"the new record was inserted")

4 the new record was inserted


### queries

In [8]:
cur.execute("SELECT * FROM students")
results = cur.fetchall()
for i in results:
    print(i)

(1, 'Omar Khaled', 'omar.khaled@example.com', '01098765432')
(2, 'Sara Nabil', 'sara.nabil@example.com', '01122334455')
(3, 'Mahmoud Adel', 'mahmoud.adel@example.com', '01233445566')
(4, 'Nour El-Sayed', 'nour.elsayed@example.com', '01044556677')
(5, 'Youssef Hassan', 'youssef.hassan@example.com', '01577889900')


### finding students with the grade "A" using thier ID

In [9]:
cur.execute("SELECT * FROM enrollments WHERE grade = 'A' ")
results = cur.fetchall()
for i in results:
    print(i)

(1, 1, 1, 'A')
(3, 2, 3, 'A')


### finding students with the grade "A" with thier names

In [10]:
cur.execute("""
SELECT DISTINCT s.student_id,s.name,e.grade
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
WHERE e.grade = 'A'
""")
rows = cur.fetchall()
for row in rows:
    print(row)

(1, 'Omar Khaled', 'A')
(2, 'Sara Nabil', 'A')


### finding credit hours of all courses

In [11]:
cur.execute("SELECT * FROM courses")
results = cur.fetchall()
for i in results:
    print(i)

(1, 'Mathematics', 3)
(2, 'Mathematics', 3)
(3, 'Computer Science', 4)
(4, 'History', 2)


### find courses where a specific student is enrolled in

In [12]:
student_name = "Omar Khaled"

cur.execute(""" 
            SELECT s.name AS student_name,c.course_name,c.credits,e.grade
            FROM students s
            JOIN enrollments e ON s.student_id=e.student_id
            JOIN courses c ON e.course_id=c.course_id
            WHERE s.name=%s
    """,(student_name,)
)
results = cur.fetchall()
for i in results:
    print(i)

('Omar Khaled', 'Mathematics', 3, 'A')
('Omar Khaled', 'Mathematics', 3, 'B')


### list each student with the courses they enlisted in

In [13]:
cur.execute("""
SELECT s.student_id, s.name, COUNT(e.course_id) AS course_count
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
GROUP BY s.student_id, s.name
ORDER BY course_count DESC
""")

rows = cur.fetchall()
for row in rows:
    print(row)



(1, 'Omar Khaled', 2)
(3, 'Mahmoud Adel', 1)
(2, 'Sara Nabil', 1)
(5, 'Youssef Hassan', 0)
(4, 'Nour El-Sayed', 0)


### find students who are not yet enrolled in any course

In [14]:
cur.execute("""
SELECT s.student_id, s.name, s.email, s.phone
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
WHERE e.enrollment_id IS NULL
""")

rows = cur.fetchall()
for row in rows:
    print(row)


(5, 'Youssef Hassan', 'youssef.hassan@example.com', '01577889900')
(4, 'Nour El-Sayed', 'nour.elsayed@example.com', '01044556677')


### show students and their enrollments sorted by grades

In [15]:
cur.execute("""
SELECT e.enrollment_id, s.name AS student_name, c.course_name, e.grade
FROM enrollments e
JOIN students s ON e.student_id = s.student_id
JOIN courses c ON e.course_id = c.course_id
ORDER BY e.grade 
""")

rows = cur.fetchall()
for row in rows:
    print(row)


(1, 'Omar Khaled', 'Mathematics', 'A')
(3, 'Sara Nabil', 'Computer Science', 'A')
(2, 'Omar Khaled', 'Mathematics', 'B')
(4, 'Mahmoud Adel', 'Mathematics', 'C')


In [17]:
grade_to_gpa = {
    'A': 4.0,
    'B': 3.0,
    'C': 2.0,
    'D': 1.0,
    'F': 0.0
}
cur.execute("""
SELECT c.course_name, e.grade
FROM enrollments e
JOIN courses c ON e.course_id = c.course_id
""")

rows = cur.fetchall()
for row in rows:
    print(row)

('Mathematics', 'A')
('Mathematics', 'B')
('Computer Science', 'A')
('Mathematics', 'C')


In [21]:
course_grades = {}
for course, grade in rows:
    gpa = grade_to_gpa.get(grade, None)  # None if grade not found
    if gpa is not None:
        course_grades.setdefault(course, []).append(gpa)
print("\nAverage GPA per course:")
for course, gpas in course_grades.items():
    avg_gpa = sum(gpas) / len(gpas)
    print(f"{course}: {avg_gpa:.2f}")


Average GPA per course:
Mathematics: 3.00
Computer Science: 4.00
